In [15]:
from torch.utils.data import DataLoader, Dataset
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoConfig, RobertaTokenizerFast
from torch.utils.data import TensorDataset, DataLoader
device = torch.device("cuda")


In [2]:
path_to_data="../data"
model_name="distilroberta-base"
ground_truth="risk_golden_truth.txt"
processed_folder="processed"

labels = []
ext = ".txt"
path_to_processed = os.path.join(path_to_data, processed_folder)
path_labels = os.path.join(path_to_data, ground_truth)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
with open(path_labels, "r") as f:
    for line in f:
        subject, label = line.split()
        labels.append((subject, label))
    # print(labels)

In [4]:
subject, label = labels[0]
subject_path = os.path.join(path_to_processed, subject) + ext
print(subject_path)

../data/processed/subject3450.txt


In [5]:
# with open(subject_path, "r") as f:
#     text = " ".join(f.readlines())
# text = tokenizer.encode(
#     text,
#     add_special_tokens=True,
#     # max_length=512,
#     padding=True,
#     return_tensors='pt',
#     truncation=True,
# )
# text


In [29]:
class BaseDataset(Dataset):
    def __init__(
        self,
        path_to_data="../data",
        model_name="distilroberta-base",
        ground_truth="risk_golden_truth.txt",
        processed_folder="processed",
    ):
        self.labels = []
        self.ext = ".txt"

        self.path_to_data = path_to_data
        self.path_to_processed = os.path.join(self.path_to_data, processed_folder)
        self.path_labels = os.path.join(self.path_to_data, ground_truth)

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        with open(self.path_labels, "r") as f:
            for line in f:
                subject, label = line.split()
                self.labels.append((subject, label))

    def __getitem__(self, idx):
        subject, label = self.labels[idx]
        subject_path = os.path.join(self.path_to_processed, subject) + self.ext
        with open(subject_path, "r") as f:
            text = " ".join(f.readlines())

        text = self.tokenizer(
            text,
            padding=True,
            truncation=True,
            return_tensors='pt',
        )
        return {
            "ids": text['input_ids'],
            "mask": text["attention_mask"],
            "labels": torch.tensor(int(label), dtype=torch.float),
        }

    def __len__(self):
        return len(self.labels)


dl = DataLoader(BaseDataset(), batch_size=1)

In [12]:
# config = AutoConfig.from_pretrained('distilroberta-base', num_labels=1)
model = AutoModelForSequenceClassification.from_pretrained('distilroberta-base', num_labels=1)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.bias

In [30]:
for x in dl:
    ids, labels = x['ids'], x['labels']
    print(ids.shape)
    # output =  model(ids)
    break

torch.Size([1, 1, 512])


In [66]:
torch.sigmoid(output.logits)

tensor([[0.4732],
        [0.4730]], grad_fn=<SigmoidBackward0>)

In [34]:
model2 = AutoModel.from_pretrained('distilroberta-base')

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
for batch in data_loader:
    ids, mask, labels = batch["ids"], batch["mask"], batch["labels"]
    out = model(ids, mask).logits
    break

In [8]:
model.classifier.out_proj.out_features=1

In [11]:
import torch

In [12]:
torch.nn.functional.softmax(out, dim=1)

tensor([[0.5897, 0.4103]], grad_fn=<SoftmaxBackward0>)

In [16]:
model.roberta(ids,mask)[0][:,0].shape

torch.Size([1, 768])